# Going to demonstrate *"chaining"* the binomial option pricing method over an arbitrary timeframe to show how it can approximate Black-Scholes.

But first, I need to figure out how to embed an image in this notebook. 

![title](multiperiodexample.png)